In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import awkward as ak
from dask.distributed import Client
#plotting
import matplotlib.pyplot as plt
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from analysis.tools import ffschema, sidm_processor
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)

<module 'analysis.tools.sidm_processor' from '/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/sidm_processor.py'>

In [2]:
from dask.distributed import Client

client = Client("tls://bryan-2ecardwell-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")

In [3]:
fileset = {
    "2Mu2E_100GeV_1p2GeV_9p6mm": [
        "root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root",
    ],
}

runner = processor.Runner(
    executor=processor.IterativeExecutor(),
    #executor=processor.FuturesExecutor(),
    #executor=processor.DaskExecutor(client=client),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "4mu",
        "2mu2e",
    ],
    ["base"]
)

# test if processor is serializable
import coffea.util as coffea_util
import pickle
coffea_util.save(p, "processor.coffea")
print(coffea_util.load("processor.coffea"))
with open("test.pickle", "wb") as f:
    pickle.dump(p, f)
with open("test.pickle", "rb") as f:
    print(pickle.load(f))

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)

Output()

Output()

In [4]:
out = output["out"]

In [5]:
out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()

cut name        individual cut N    all cut N
------------  ------------------  -----------
No selection             11518.1      11518.1
PV filter                11518.1      11518.1
Cosmic veto              11518.1      11518.1
>=2 LJs                   1433.1       1433.1
2mu2e                     1324.4       1324.4


In [6]:
from coffea.processor import accumulate
double_out = accumulate([output["out"], output["out"]])

/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:104: RuntimeWarning: invalid value encountered in float_scalars
  self.f_ind = self.n_ind / self.n_evts
/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:105: RuntimeWarning: invalid value encountered in float_scalars
  self.f_all = self.n_all / self.n_evts
/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:108: RuntimeWarning: invalid value encountered in float_scalars
  self.f_mar = self.n_all / self.cutflow.flow[-1].n_all
/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:104: RuntimeWarning: invalid value encountered in true_divide
  self.f_ind = self.n_ind / self.n_evts
/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:105: RuntimeWarning: invalid value encountered in true_divide
  self.f_all = self.n_all / self.n_evts
/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/cutflow.py:

In [7]:
double_out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()

cut name        individual cut N    all cut N
------------  ------------------  -----------
No selection                 0.0          0.0
PV filter                23036.3      23036.3
Cosmic veto              23036.3      23036.3
>=2 LJs                   2866.1       2866.1
2mu2e                     2648.9       2648.9
